<a href="https://colab.research.google.com/github/AHartNtkn/DS-Unit-2-Regression-Classification/blob/master/module2/DS7_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [X] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [X] Engineer at least two new features. (See below for explanation & ideas.)
- [X] Fit a linear regression model with at least two features.
- [X] Get the model's coefficients and intercept.
- [X] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [X] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [X] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Do the [Plotly Dash](https://dash.plot.ly/) Tutorial, Parts 1 & 2.
- [ ] Add your own stretch goal(s) !

In [0]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [4]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Change the date times.
df['created'] = pd.to_datetime(df['created'])

In [0]:
trainDf = df[(df['created'] >= pd.to_datetime('April 1st, 2016')) & (df['created'] < pd.to_datetime('June 1st, 2016'))]
testDf = df[(df['created'] >= pd.to_datetime('April 1st, 2016')) & (df['created'] < pd.to_datetime('July 1st, 2016'))]            

In [0]:
# Create two new features
df['description length'] = df['description'].str.len().fillna(0)
df['total perks'] = df[df.columns[10:]].T.sum()

In [0]:
# Proccess interest into a measurable quantity.
int_ranks = {'low': 0, 'medium': 1, 'high': 2}
df['interest_level'] = df['interest_level'].map(int_ranks)

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [42]:
features = ['description length', 'total perks', 'bathrooms', 'bedrooms', 'interest_level']
target = ['price']
model = LinearRegression()
model.fit(trainDf[features],trainDf[target])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [43]:
model.coef_, model.intercept_

(array([[ -82.1299242 ,   41.08380042, 1838.46105023,  421.95097152,
         -515.11832763]]), array([505.20565218]))

In [57]:
trainMAE = mean_absolute_error(trainDf[target], model.predict(trainDf[features]))
trainRMSE = mean_squared_error(trainDf[target], model.predict(trainDf[features])) ** (1/2)
trainR2 = r2_score(trainDf[target], model.predict(trainDf[features]))

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
testRMSE = mean_squared_error(testDf[target], model.predict(testDf[features])) ** (1/2)
testR2 = r2_score(testDf[target], model.predict(testDf[features]))

print('Train MAE:', round(trainMAE),
      '\nTrain RMSE:', round(trainRMSE),
      '\nTrain R^2:', round(trainR2,3))


print('\nTest MAE:', round(testMAE),
      '\nTest RMSE:', round(testRMSE),
      '\nTest R^2:', round(testR2,3))

Train MAE: 761.0 
Train RMSE: 1161.0 
Train R^2: 0.566

Test MAE: 760.0 
Test RMSE: 1155.0 
Test R^2: 0.57


In [0]:
# Add a new column for if there is a description
df['described'] = df['description length'] > 0

In [0]:
# Add a new column for if either cats or dogs are allowed
df['pets_allowed'] = (df['cats_allowed'] + df['dogs_allowed']) > 0

In [58]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=5)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))


Test MAE: 816.0


In [59]:
features = ['description length', 'total perks', 'bathrooms', 'bedrooms', 'interest_level', 'latitude', 'longitude']
model = KNeighborsRegressor(n_neighbors=5)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))


Test MAE: 793.0


In [94]:
features = ['described', 'total perks', 'bathrooms', 'bedrooms', 'interest_level', 'latitude', 'longitude']
model = KNeighborsRegressor(n_neighbors=5)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))


Test MAE: 752.0


In [65]:
features = ['bathrooms', 'bedrooms', 'latitude', 'longitude']
model = KNeighborsRegressor(n_neighbors=5)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))


Test MAE: 361.0


In [87]:
features = ['bathrooms', 'bedrooms', 'latitude', 'longitude']
model = KNeighborsRegressor(n_neighbors=1)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))


Test MAE: 256.0


In [86]:
features = ['bathrooms', 'bedrooms', 'interest_level', 'latitude', 'longitude']
model = KNeighborsRegressor(n_neighbors=1)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))


Test MAE: 236.0


In [101]:
features = ['bathrooms', 'bedrooms', 'interest_level', 'latitude', 'longitude','pets_allowed']
model = KNeighborsRegressor(n_neighbors=1)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))


Test MAE: 228.0


In [105]:
# !!!!==== Best So Far ====!!!!
features = ['bathrooms', 'bedrooms', 'interest_level', 'latitude', 'longitude','pets_allowed', 'doorman']
model = KNeighborsRegressor(n_neighbors=1)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))


Test MAE: 222.0


In [66]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(learning_rate=0.2, max_depth=6, min_samples_leaf=15, max_leaf_nodes=110)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Test MAE: 439.0


In [67]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(learning_rate=0.2, max_depth=6, min_samples_leaf=7, max_leaf_nodes=25)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Test MAE: 462.0


In [68]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(learning_rate=0.1, max_depth=6, min_samples_leaf=200, max_leaf_nodes=60)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Test MAE: 528.0


In [69]:
features = ['bathrooms', 'bedrooms', 'latitude', 'longitude', 'interest_level']
model = GradientBoostingRegressor(learning_rate=0.2, max_depth=6, min_samples_leaf=90, max_leaf_nodes=60)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Test MAE: 457.0


In [73]:
features = ['bathrooms', 'bedrooms', 'latitude', 'longitude', 'interest_level','total perks']
model = GradientBoostingRegressor(learning_rate=0.2, max_depth=6, min_samples_leaf=90, max_leaf_nodes=60)

model.fit(trainDf[features], trainDf[target])

testMAE = mean_absolute_error(testDf[target], model.predict(testDf[features]))
print('\nTest MAE:', round(testMAE))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Test MAE: 456.0


In [71]:
linreg(trainDf, testDf, features, target)

Train MAE: 445.4634642553085
test MAE: 456.5115869601907
